In [1]:
%%latex
\tableofcontents

<IPython.core.display.Latex object>

# Ambulance_Dispatch_2024_Reduce_Dimensionality

## Goals

- The goal of this notebook is to reduce the dimensionality of each dataset by identifying and dropping features that are well predicted by the remaining features by multicollinearity.

- In the next notebook we will build models on three groups of features:
    - Easy:  Features that the emergency dispatcher already has or can, without really precise maps, determine from the location
    - Medium:  Additionally, features that the emergency dispatcher can determine from precise maps and information from the cell service provider about the primary user of the phone
    - Hard:  Additionally, features about the vehicle that can be learned only by correlating information about the identity of the likely driver with vehicle registration and/or insurance records.  Not easily available in real time without lots of preparation, raises privacy concerns, and not likely to be very accurate

- The outbook of this notebook is the three sets of features, expressed as a dummy variable for each binned value of the feature, after dimensionality reduction.



## Methods

- For each of [Easy, Medium, Hard]:
    - For each feature:
        - Create a linear model (LinearRegression from sklearn) mapping the other features onto this feature.
        - Find the $R^2$ score from fitting the model:  ``r2 = LinearRegression().fit(X, y).score(X, y)`` where ``y`` is this feature and ``X`` is all of the other features.  
        - If $R^2=1$, then this feature is perfectly predicted by the other features.  
        - If this $R^2$ score is high, like greater than 0.9, then this feature is well predicted by other features, and we should consider dropping it.
        - The reason to not drop all features with high $R^2$ scores in the same step is that two (or more) features could be highly collinear, and we may only want to drop one of them.  If we drop one of them and recalculate, we may see that the other(s) now have low $R^2$ scores.

    - While the $\max(R^2) > 0.90$, run the above method and drop the feature with the highest $R^2$ score.
        - This choice of threshold of 0.90 is somewhat arbitary, perhaps arbitrarily high, and testing the results of different choices is an opportunity for future research.

    - Transform the features, which have 2-10 values, into 1-9 dummy features.  We choose to drop the first value because each of the dummy features of an original feature would be perfectly collinear with the others.

    - Repeat the ``While the $\max(R^2) > 0.90$,...'' process to reduce the number of features.  
    
    - Write the reduced dummy-variable features to file to use in the next notebook, where we will build models predicting whether each crash person needs an ambulance.
    
- We considered using Principal Component Analysis, but decided on $R^2$ instead.


# Setup

## Import Libraries

In [2]:
print ('Install Packages')

import sys, copy, math, time, os, csv

print ('Python version: {}'.format(sys.version))

import numpy as np
print ('NumPy version: {}'.format(np.__version__))
np.set_printoptions(suppress=True)

import pandas as pd
print ('Pandas version:  {}'.format(pd.__version__))
pd.set_option('display.max_rows', 500)

import sklearn
print ('sklearn version: {}'.format(sklearn.__version__))
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Set Randomness.  
import random
random_seed = 0
print ('random_seed = ', random_seed)
np.random.seed(random_seed) # NumPy
random.seed(random_seed) # Python
#tf.random.set_seed(42) # Tensorflow

import warnings
warnings.filterwarnings('ignore')

print ('Finished Installing Packages')

Install Packages
Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:51:49) [Clang 16.0.6 ]
NumPy version: 1.26.4
Pandas version:  2.2.2
sklearn version: 1.5.0
random_seed =  0
Finished Installing Packages


## Get Data

This function pulls in the saved output from Ambulance_Dispatch_2024_03_Impute_Missing_Data.

In [3]:
def Get_Data(target):
    print ('Get_Data')
    
    # data imputed by MissForest
#    data = pd.read_csv('../../Big_Files/CRSS_03_Imputed_by_MF_Data.csv', index_col=None) 
    # data imputed by IVEware
#    data = pd.read_csv('../../Big_Files/CRSS_03_Imputed_by_IVEware_seed_0.csv', index_col=None)

    filename = '../../Big_Files/CRSS_03' + Run[:6] + '.csv'
    print (filename)
    print ()
    data = pd.read_csv(filename, index_col=None)
    
    print ('data.shape = ', data.shape)
    data = data.reindex(sorted(data.columns), axis=1)
    first_column = data.pop(target) 
    data.insert(0, target, first_column)     

    return data

#data =  Get_Data('HOSPITAL')

In [4]:
def Thin_to_Hard_Features(data):
    print ('Thin_to_Hard_Features()')

    Merge = [
        'CASENUM',
        'VEH_NO',
        'PER_NO',        
    ]

    Accident = [
        'DAY_WEEK',
        'HOUR',
        'INT_HWY',
        'LGT_COND',
        'MONTH',
#        'PEDS',
        'PERMVIT',
#        'PERNOTMVIT', # Pedestrians, which we have taken out
        'PJ',
        'PSU',
        'PVH_INVL',
        'REGION',
        'REL_ROAD',
        'RELJCT1',
        'RELJCT2',
#        'SCH_BUS',
        'TYP_INT',
        'URBANICITY',
        'VE_FORMS',
        'VE_TOTAL',
        'WEATHER',
        'WRK_ZONE',
#        'YEAR',
    ]
    
    Vehicle = [
        'BODY_TYP',
#        'BUS_USE',
#        'EMER_USE',
        'MAKE',
#        'MOD_YEAR',
        'MODEL',
        'NUMOCCS',
        'VALIGN',
#        'VNUM_LAN',
        'VPROFILE',
        'VSPD_LIM',
#        'VSURCOND',
        'VTRAFCON',
        'VTRAFWAY',
    ]
    
    Person = [
        'AGE',
#        'LOCATION', # Pedestrian location; taken out
        'PER_TYP',
        'SEX',
        'HOSPITAL',    
    ]

    Engineered = [
        'VEH_AGE',
    ]
    
    # Put features in alphabetical order
    Features = Accident + Vehicle + Person + Engineered
    Features = sorted(Features)

    print ('Removed Features')
    for feature in data:
        if feature not in Features:
            print (feature)
    print ()
    
    data = data.filter(Features, axis=1)
    
    print ('data.shape: ', data.shape)
    
    print ('End Thin_to_Hard_Features()')
    print ()
        
    return data

def Test_Thin_to_Hard_Features():
    data = Get_Data()
    data = Thin_to_Hard_Features(data)
    for feature in data:
        display(data[feature].value_counts())
        
#Test_Thin_to_Hard_Features()

In [5]:
def Thin_to_Medium_Features(data):
    print ('Thin_to_Medium_Features()')

    Accident = [
        'DAY_WEEK',
        'HOUR',
        'INT_HWY',
#        'LGT_COND',
        'MONTH',
#        'PEDS',
#        'PERMVIT',
#        'PERNOTMVIT',
        'PJ',
        'PSU',
#        'PVH_INVL',
        'REGION',
        'REL_ROAD',
        'RELJCT1',
#        'RELJCT2',
#        'SCH_BUS',
        'TYP_INT',
        'URBANICITY',
#        'VE_FORMS',
#        'VE_TOTAL',
        'WEATHER',
#        'WRK_ZONE',
#        'YEAR',
    ]
    
    Vehicle = [
#        'BODY_TYP',
#        'BUS_USE',
#        'EMER_USE',
#        'MAKE',
#        'MOD_YEAR',
#        'MODEL',
#        'NUMOCCS',
        'VALIGN',
#        'VNUM_LAN',
        'VPROFILE',
        'VSPD_LIM',
#        'VSURCOND',
        'VTRAFCON',
        'VTRAFWAY',
    ]
    
    Person = [
        'AGE',
#        'LOCATION',
#        'PER_TYP',
        'SEX',
        'HOSPITAL',    
    ]

    Engineered = [
#        'VEH_AGE',
    ]
    
    # Put features in alphabetical order
    Features = Accident + Vehicle + Person + Engineered
    Features = sorted(Features)
    
    print ('Removed Features')
    for feature in data:
        if feature not in Features:
            print (feature)
    print ()
    
    data = data.filter(Features, axis=1)
    
    print ('data.shape: ', data.shape)
    
    print ('End Thin_to_Medium_Features()')
    print ()
        
    return data

def Test_Thin_to_Medium_Features():
    data = Get_Data()
    data = Thin_to_Medium_Features(data)
    for feature in data:
        display(data[feature].value_counts())
        
#Test_Thin_to_Medium_Features()

In [6]:
def Thin_to_Easy_Features(data):
    print ('Thin_to_Easy_Features()')

    Accident = [
        'DAY_WEEK',
        'HOUR',
#        'INT_HWY',
#        'LGT_COND',
        'MONTH',
#        'PEDS',
#        'PERMVIT',
#        'PERNOTMVIT',
        'PJ',
        'PSU',
#        'PVH_INVL',
        'REGION',
#        'REL_ROAD',
#        'RELJCT1',
#        'RELJCT2',
#        'SCH_BUS',
#        'TYP_INT',
        'URBANICITY',
#        'VE_FORMS',
#        'VE_TOTAL',
        'WEATHER',
#        'WRK_ZONE',
#        'YEAR',
    ]
    
    Vehicle = [
#        'BODY_TYP',
#        'BUS_USE',
#        'EMER_USE',
#        'MAKE',
#        'MOD_YEAR',
#        'MODEL',
#        'NUMOCCS',
#        'VALIGN',
#        'VNUM_LAN',
#        'VPROFILE',
#        'VSPD_LIM',
#        'VSURCOND',
#        'VTRAFCON',
#        'VTRAFWAY',
    ]
    
    Person = [
#        'AGE',
#        'LOCATION',
#        'PER_TYP',
#        'SEX',
        'HOSPITAL',    
    ]

    Engineered = [
#        'VEH_AGE',
#        'AGE_x_SEX',
#        'AGE_x_SCH_BUS'
    ]
    
    # Put features in alphabetical order
    Features = Accident + Vehicle + Person + Engineered
    Features = sorted(Features)

    print ('Removed Features')
    for feature in data:
        if feature not in Features:
            print (feature)
    print ()
        
    data = data.filter(Features, axis=1)
    
    print ('data.shape: ', data.shape)
    
    print ('End Thin_to_Easy_Features()')
    print ()
        
    return data

def Test_Thin_to_Easy_Features():
    data = Get_Data()
    data = Thin_to_Easy_Features(data)
    for feature in data:
        display(data[feature].value_counts())
        
#Test_Thin_to_Easy_Features()

In [7]:
def Get_Dummies(data, target):
    print ('Get_Dummies')
    print (data.shape)
    data = data.astype('category')
    Target = data.pop(target)
    
    data_Dummies = pd.get_dummies(data, prefix = data.columns, drop_first = True)

    # Use this version if the dataset has "99" signifying "Missing/Unknown",
    # but not if missing values have already been imputed.
#    data_Dummies = pd.get_dummies(data, prefix = data.columns, drop_first = False)
#    for feature in data_Dummies:
#        if '99' in feature:
#            data_Dummies.drop(columns=[feature], inplace=True)

    data_Dummies = data_Dummies.join(Target)
#    for feature in data_Dummies:
#        print (feature)
    print (data_Dummies.shape)
    print ()
    

    return data_Dummies

#data = Get_Dummies(data, 'HOSPITAL')

In [8]:
def Principal_Component_Analysis(data):
    print ('Principal_Component_Analysis()')
    Features = [feature for feature in data]
    n_components=300
    pca = PCA(n_components)
    print ('pca.fit()')
    pca.fit(data)
    
    data_pca = pca.transform(data)
    print (data_pca.shape)
    print (data_pca[:10])
    print ()
    data_pca = np.ascontiguousarray(data_pca)
    print (data_pca.shape)
    print (data_pca[:10])
    print ()
    data_pca = pd.DataFrame(data_pca) #, columns=['PCA%i' % i for i in range(n_components)], index=data.index)
    print (data_pca.head())
    print (data_pca.shape)
    print ()
    

    return data_pca

In [9]:
# Adapted from https://towardsdatascience.com/statistics-in-python-collinearity-and-multicollinearity-4cc4dcd82b3f
def calculate_vif(df, features):  
    print ('calculate_vif()')
    r2_Dict, tolerance, vif = {}, {}, {}
    # all the features that you want to examine
    for feature in features:
        # extract all the other features you will regress against
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]
        # extract r-squared from the fit
        r2 = LinearRegression().fit(X, y).score(X, y)
        r2_Dict[feature] = r2
#        print ('Built LinearRegression model with target', feature, ' that has score  R^2 = ', round(r2,2))
        # calculate tolerance
        tolerance[feature] = 1 - r2
        # calculate VIF
        if tolerance[feature] !=0:
            vif[feature] = 1/(tolerance[feature])
        else:
            vif[feature] = 10000

    return pd.DataFrame({'r2': r2_Dict, 'Tolerance': tolerance, 'VIF': vif}), tolerance, r2_Dict

In [10]:
# Iteratively remove the feature with the largest VIF ('Variance Inflaction Factor')
# until the largest VIF is 10, or smallest Tolerance is 0.1, or largest R^2 is 0.9

# This process could be much more efficient if, whenever we got R^2==1.0,
    # we dropped the feature immediately and started the calculate_vif() over.
def Reduce_Dimensionality(data, target):
    Target = data.pop(target)
    Features = [feature for feature in data]
    VIF, Tolerance_Dict, r2_Dict = calculate_vif(data, Features)
    Max_r2_Feature = VIF['r2'].idxmax()
    display(VIF)
    print (Max_r2_Feature)
    
    Dropped_Features = []
    
    if r2_Dict[Max_r2_Feature] > 0.9:
        data.drop(columns = [Max_r2_Feature], inplace=True)
        print ('Drop ', Max_r2_Feature)
        Dropped_Features.append(Max_r2_Feature)
    print ()
    while r2_Dict[Max_r2_Feature] > 0.9:
        Features = [feature for feature in data]
        VIF, Tolerance_Dict, r2_Dict = calculate_vif(data, Features)
        Max_r2_Feature = VIF['r2'].idxmax()
        display(VIF)
        print (Max_r2_Feature)
        if r2_Dict[Max_r2_Feature] > 0.9:
            data.drop(columns = [Max_r2_Feature], inplace=True)
            print ('Drop ', Max_r2_Feature)
            Dropped_Features.append(Max_r2_Feature)
        print ()

    data = data.join(Target)
    
    return data, Dropped_Features
        
    

In [11]:
%%time
# About two hours
def Main():
    target = 'HOSPITAL'

    for i in range (3):
        Level = ['_Easy', '_Medium', '_Hard']
        data = Get_Data(target)
        data = data.astype('int64')
        if i==2:
            print ('Thin_to_Hard_Features()')
            data = Thin_to_Hard_Features(data)
        if i==1:
            print ('Thin_to_Medium_Features')
            data = Thin_to_Medium_Features(data)
        if i==0:
            print ('Thin_to_Easy_Features')
            data = Thin_to_Easy_Features(data)
            
#    Features = [feature for feature in data]
#    VIF, VIF_Dict = calculate_vif(data, Features)
#    display(VIF)
#    print ()


        if Run[7]=='1': 
            data, Dropped_Features = Reduce_Dimensionality(data, target)
            # Write Dropped_Features to csv
            filename = './Analysis/CRSS_04_Dropped_Features' + Run + Level[i] + '.csv'
            print (filename)
            print ()
            f = open(filename, 'w')
            for feature in Dropped_Features:
                f.write('%s\n' % (feature))
        
    
        for feature in data:
            data[feature] = pd.to_numeric(data[feature])
        print (data.shape)

        data_dummies = Get_Dummies(data, target)
#        for feature in data_dummies:
#            print (feature)
        print (data_dummies.shape)
        print ()
        
        """
        if Run[7]==1: 
            data_dummies, Dropped_Features = Reduce_Dimensionality(data_dummies, target)
            filename = './Analysis/CRSS_04_Dropped_Features' + Run + Level[i] + '.csv'
            print (filename)
            print ()
            f = open(filename, 'a')
            for feature in Dropped_Features:
                f.write('%s\n' % (feature))
            f.close()
        """

#        for feature in data_dummies:
#            print (feature)
        print (data_dummies.shape)
        print ()
        
        
        if i==2:
            filename = '../../Big_Files/CRSS_04' + Run + '_Hard.csv'
            data_dummies.to_csv(filename, index=False)
            print ()
            print (filename)
            print ()
        if i==1:
            filename = '../../Big_Files/CRSS_04' + Run + '_Medium.csv'
            data_dummies.to_csv(filename, index=False)
            print ()
            print (filename)
            print ()
        if i==0:
            filename = '../../Big_Files/CRSS_04' + Run + '_Easy.csv'
            data_dummies.to_csv(filename, index=False)
            print ()
            print (filename)
            print ()
    
    print ()
    print ('Finished!')
    
    
#Main()

CPU times: user 5 μs, sys: 1 μs, total: 6 μs
Wall time: 9.78 μs


In [12]:
Run = ''
Runs = [
#    '_0_0_0_0',
#    '_0_0_0_1',
    '_0_0_1_0',
    '_0_0_1_1',
    '_0_1_0_0',
    '_0_1_0_1',
    '_0_1_1_0',
    '_0_1_1_1',
    '_1_0_0_0',
    '_1_0_0_1',
    '_1_0_1_0',
    '_1_0_1_1',
    '_1_1_0_0',
    '_1_1_0_1',
    '_1_1_1_0',
    '_1_1_1_1',
]

for run in Runs:
    Run = run
    print ()
    print ('------------------------------------')
    print ('Run = ', Run)

    random_seed = int(Run[1])
    print ('random_seed = ', random_seed)
    random.seed(random_seed) # Python
    np.random.seed(random_seed) # NumPy
    #tf.random.set_seed(42) # Tensorflow
    
    Main()
    print ()



------------------------------------
Run =  _0_0_1_0
random_seed =  0
Get_Data
../../Big_Files/CRSS_03_0_0_1.csv

data.shape =  (802700, 68)
Thin_to_Easy_Features
Thin_to_Easy_Features()
Removed Features
ACC_TYPE
AGE
AIR_BAG
ALC_STATUS
BODY_TYP
CARGO_BT
DEFORMED
DR_ZIP
EJECTION
HARM_EV
HIT_RUN
IMPACT1
INJ_SEV
INT_HWY
J_KNIFE
LGT_COND
MAKE
MAK_MOD
MAN_COLL
MAX_SEV
MAX_VSEV
MODEL
M_HARM
NUMOCCS
NUM_INJ
NUM_INJV
PCRASH4
PCRASH5
PERMVIT
PER_TYP
PVH_INVL
P_CRASH1
P_CRASH2
RELJCT1
RELJCT2
REL_ROAD
REST_MIS
REST_USE
ROLINLOC
ROLLOVER
SEAT_POS
SEX
SPEC_USE
SPEEDREL
TOWED
TOW_VEH
TYP_INT
Unnamed: 0
VALIGN
VEH_AGE
VE_FORMS
VE_TOTAL
VPROFILE
VSPD_LIM
VSURCOND
VTCONT_F
VTRAFCON
VTRAFWAY
WRK_ZONE

data.shape:  (802700, 9)
End Thin_to_Easy_Features()

(802700, 9)
Get_Dummies
(802700, 9)
(802700, 40)

(802700, 40)

(802700, 40)


../../Big_Files/CRSS_04_0_0_1_0_Easy.csv

Get_Data
../../Big_Files/CRSS_03_0_0_1.csv

data.shape =  (802700, 68)
Thin_to_Medium_Features
Thin_to_Medium_Features()
Removed F

,r2,Tolerance,VIF
DAY_WEEK,0.001919,0.998081,1.001923
HOUR,0.003341,0.996659,1.003352
MONTH,0.001533,0.998467,1.001535
PJ,0.544922,0.455078,2.197425
PSU,0.558646,0.441354,2.265757
REGION,0.054948,0.945052,1.058143
URBANICITY,0.021225,0.978775,1.021686
WEATHER,0.007372,0.992628,1.007427


PSU

./Analysis/CRSS_04_Dropped_Features_0_0_1_1_Easy.csv

(802700, 9)
Get_Dummies
(802700, 9)
(802700, 40)

(802700, 40)

calculate_vif()


,r2,Tolerance,VIF
DAY_WEEK_1,0.265204,0.734796,1.360922
DAY_WEEK_2,0.274802,0.725198,1.378934
DAY_WEEK_3,0.368850,0.631150,1.584410
DAY_WEEK_4,0.292632,0.707368,1.413691
HOUR_1,0.507626,0.492374,2.030978
HOUR_2,0.520898,0.479102,2.087237
HOUR_3,0.726530,0.273470,3.656703
HOUR_4,0.870921,0.129079,7.747190
HOUR_5,0.873106,0.126894,7.880572
HOUR_6,0.790258,0.209742,4.767765


PSU_3

./Analysis/CRSS_04_Dropped_Features_0_0_1_1_Easy.csv

(802700, 40)


../../Big_Files/CRSS_04_0_0_1_1_Easy.csv

Get_Data
../../Big_Files/CRSS_03_0_0_1.csv

data.shape =  (802700, 68)
Thin_to_Medium_Features
Thin_to_Medium_Features()
Removed Features
ACC_TYPE
AIR_BAG
ALC_STATUS
BODY_TYP
CARGO_BT
DEFORMED
DR_ZIP
EJECTION
HARM_EV
HIT_RUN
IMPACT1
INJ_SEV
J_KNIFE
LGT_COND
MAKE
MAK_MOD
MAN_COLL
MAX_SEV
MAX_VSEV
MODEL
M_HARM
NUMOCCS
NUM_INJ
NUM_INJV
PCRASH4
PCRASH5
PERMVIT
PER_TYP
PVH_INVL
P_CRASH1
P_CRASH2
RELJCT2
REST_MIS
REST_USE
ROLINLOC
ROLLOVER
SEAT_POS
SPEC_USE
SPEEDREL
TOWED
TOW_VEH
Unnamed: 0
VEH_AGE
VE_FORMS
VE_TOTAL
VSURCOND
VTCONT_F
WRK_ZONE

data.shape:  (802700, 20)
End Thin_to_Medium_Features()

calculate_vif()


...

PERMVIT_3
Drop  PERMVIT_3

calculate_vif()


,r2,Tolerance,VIF
AGE_1,0.169417,0.830583,1.203974
AGE_2,0.115014,0.884986,1.129962
AGE_3,0.045897,0.954103,1.048105
AGE_4,0.028371,0.971629,1.029200
AGE_5,0.045042,0.954958,1.047166
AGE_7,0.091875,0.908125,1.101170
AGE_8,0.049949,0.950051,1.052575
AGE_9,0.055429,0.944571,1.058682
BODY_TYP_1,0.099011,0.900989,1.109892
BODY_TYP_3,0.156641,0.843359,1.185735


HOUR_4
Drop  HOUR_4

calculate_vif()


,r2,Tolerance,VIF
AGE_1,0.169236,0.830764,1.203711
AGE_2,0.114942,0.885058,1.129870
AGE_3,0.045896,0.954104,1.048104
AGE_4,0.028333,0.971667,1.029160
AGE_5,0.045005,0.954995,1.047126
AGE_7,0.091268,0.908732,1.100435
AGE_8,0.049076,0.950924,1.051608
AGE_9,0.054070,0.945930,1.057160
BODY_TYP_1,0.098968,0.901032,1.109839
BODY_TYP_3,0.156640,0.843360,1.185734


PSU_3

./Analysis/CRSS_04_Dropped_Features_1_1_1_1_Hard.csv

(802700, 129)


../../Big_Files/CRSS_04_1_1_1_1_Hard.csv


Finished!



In [15]:
%%time
run = '_0_0_1_0_1'
Run = run
random_seed = int(run[1])
Main()

Get_Data
../../Big_Files/CRSS_03_0_0_1.csv

data.shape =  (802700, 68)
Thin_to_Easy_Features
Thin_to_Easy_Features()
Removed Features
ACC_TYPE
AGE
AIR_BAG
ALC_STATUS
BODY_TYP
CARGO_BT
DEFORMED
DR_ZIP
EJECTION
HARM_EV
HIT_RUN
IMPACT1
INJ_SEV
INT_HWY
J_KNIFE
LGT_COND
MAKE
MAK_MOD
MAN_COLL
MAX_SEV
MAX_VSEV
MODEL
M_HARM
NUMOCCS
NUM_INJ
NUM_INJV
PCRASH4
PCRASH5
PERMVIT
PER_TYP
PVH_INVL
P_CRASH1
P_CRASH2
RELJCT1
RELJCT2
REL_ROAD
REST_MIS
REST_USE
ROLINLOC
ROLLOVER
SEAT_POS
SEX
SPEC_USE
SPEEDREL
TOWED
TOW_VEH
TYP_INT
Unnamed: 0
VALIGN
VEH_AGE
VE_FORMS
VE_TOTAL
VPROFILE
VSPD_LIM
VSURCOND
VTCONT_F
VTRAFCON
VTRAFWAY
WRK_ZONE

data.shape:  (802700, 9)
End Thin_to_Easy_Features()

(802700, 9)
Get_Dummies
(802700, 9)
(802700, 40)

(802700, 40)

(802700, 40)


../../Big_Files/CRSS_04_0_0_1_0_1_Easy.csv

Get_Data
../../Big_Files/CRSS_03_0_0_1.csv

data.shape =  (802700, 68)
Thin_to_Medium_Features
Thin_to_Medium_Features()
Removed Features
ACC_TYPE
AIR_BAG
ALC_STATUS
BODY_TYP
CARGO_BT
DEFORMED
DR_ZIP